## Implementação de modelo de machine learn para classificação de perfis de candidatos a prefeito eleitos e não eleitos.

#### Esse script tem por objetivo, a partir de uma base já tratada, avaliar o resultado de diferentes algoritmos de machine learning para classificação de perfis de candidatos a prefeito no Brasil nas eleições de 2016 e 2020.

#### Trabalho de Conclusão de Curso apresentado ao Curso de Especialização em Inteligência Artificial e Aprendizado de Máquina como requisito parcial à obtenção do título de especialista. Adma Raia

In [1]:
# Instalação de pacotes
!pip install pydotplus
!pip install dtreeviz
!pip install pandas_profiling
!pip install joblib

     |████████████████████████████████| 61kB 3.0MB/s 
  Created wheel for dtreeviz: filename=dtreeviz-1.3-cp37-none-any.whl size=66642 sha256=e22667c2bb2d9d2931f2f5cf8cede4e57f37d5a2ddeb75d05b07118723faaf10
  Stored in directory: /root/.cache/pip/wheels/60/36/b1/188ee35c677e48463f6482d580f81c19f5f82ae5adbe293fd8
Successfully built dtreeviz


In [3]:
# Carregamento de conjunto de bibliotecas

import pandas as pd
import os
import numpy as np
import pydotplus 


import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
% matplotlib inline

# Carregamento das bibliotecas dos modelos
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import tree

# Carregamento das bibliotecas de processamento
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib

# Carregamento das bibliotecas de avaliação
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn import metrics
from pandas_profiling import ProfileReport
from sklearn.model_selection import cross_val_score


/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


### Configuração de tela (truncamento de linhas e colunas)

In [4]:
pd.options.display.max_columns = 500
pd.options.display.max_rows = 500

# Carregamento dataset

In [5]:
from google.colab import files
uploaded = files.upload()
base = pd.read_csv("base_treeBR.csv")

Saving base_treeBR.csv to base_treeBR.csv


# Etapa de preparação/transformação dos dados para utilização dos modelos.


In [6]:
# Verificando se as features foram carregadas com categories
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34870 entries, 0 to 34869
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ANO_ELEICAO           34870 non-null  int64 
 1   GENERO                34870 non-null  object
 2   GRAU_INSTRUCAO2       34870 non-null  object
 3   ESTADO_CIVIL2         34870 non-null  object
 4   COR_RACA2             34870 non-null  object
 5   FAIXA_ETARIA          34870 non-null  object
 6   CIDNAS_IGUAL_CIDCAND  34870 non-null  object
 7   SITUACAO_FINAL        34870 non-null  object
dtypes: int64(1), object(7)
memory usage: 2.1+ MB


In [7]:
# Transformando as features em category
for c in base.columns[base.dtypes == object]: # df.dtypes == 'object'
    base[c] = base[c].astype('category')
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34870 entries, 0 to 34869
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   ANO_ELEICAO           34870 non-null  int64   
 1   GENERO                34870 non-null  category
 2   GRAU_INSTRUCAO2       34870 non-null  category
 3   ESTADO_CIVIL2         34870 non-null  category
 4   COR_RACA2             34870 non-null  category
 5   FAIXA_ETARIA          34870 non-null  category
 6   CIDNAS_IGUAL_CIDCAND  34870 non-null  category
 7   SITUACAO_FINAL        34870 non-null  category
dtypes: category(7), int64(1)
memory usage: 511.9 KB


In [8]:
print("\n Campos do dataset:\n{0}\n".format(list(base.keys())))


 Campos do dataset:
['ANO_ELEICAO', 'GENERO', 'GRAU_INSTRUCAO2', 'ESTADO_CIVIL2', 'COR_RACA2', 'FAIXA_ETARIA', 'CIDNAS_IGUAL_CIDCAND', 'SITUACAO_FINAL']



In [9]:
# Grau de instrução é categorico ordinal, então necessita realizar o ajuste
base.GRAU_INSTRUCAO2.cat.reorder_categories(['LÊ E ESCREVE','FUNDAMENTAL', 'MÉDIO', 'SUPERIOR'], ordered=True, inplace=True)
base.GRAU_INSTRUCAO2

0           SUPERIOR
1           SUPERIOR
2              MÉDIO
3              MÉDIO
4           SUPERIOR
            ...     
34865       SUPERIOR
34866    FUNDAMENTAL
34867          MÉDIO
34868       SUPERIOR
34869       SUPERIOR
Name: GRAU_INSTRUCAO2, Length: 34870, dtype: category
Categories (4, object): ['LÊ E ESCREVE' < 'FUNDAMENTAL' < 'MÉDIO' < 'SUPERIOR']

In [10]:
#Modificando as demais categorias, mas ele ja vai considerar as categorias reordenadas
for c in base.columns[base.dtypes == object]: # df.dtypes == 'object'
    base[c] = base[c].astype('category')

In [12]:
#Codificando as categorias
for c in base.columns[base.dtypes == 'category']: # df.dtypes == 'object'
    base[c] = base[c].cat.codes

# Separação do conjunto de dados de treino e teste

In [13]:
# Separação dos atributos para classificação e  Alvo
X = base.iloc[:,1:7]
y = base.iloc[:,7]

In [14]:
# Separação de treino e teste. 
# os parâmetros utilizados para realizar a separação dos dados foram: random_state=42 para comecar sempre no mesmo ponto, neste caso por opção 42;  
# shuffle=True para embaralhar os dados; stratify = y para estratificar o atributo y, com objetivo de minimizar o impacto de desbalanceamento da classe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, shuffle=True, stratify=y)

In [15]:
# Verificar o shape dos dados separados em treino e teste
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((26152, 6), (8718, 6), (26152,), (8718,))

In [16]:
# Normalizar os dados
scaler = MinMaxScaler()
X_norm = scaler.fit_transform(X)

In [17]:
# Verificando os atributos alvo, observa-se que há desbalanceamento.
target_count = base.iloc[:,-1].value_counts()
print(target_count)

print("y=1  (Não Eleito)  ", (y==1).sum())
print("y=0  (  Eleito  )  ",(y==0).sum())

1    23817
0    11053
Name: SITUACAO_FINAL, dtype: int64
y=1  (Não Eleito)   23817
y=0  (  Eleito  )   11053


# Etapa de implementação dos modelos e avaliação
## 1 - KNN;
## 2 - Decision Tree;
## 3 - Random Forest;
## 4 - Support Vector Machine

# 1 - KNN 
### no método é determinado o rótulo de classificação de uma amostra baseado nas amostras vizinhas, oriundas de um conjunto de treinamento (Faceli et al., 2015).

In [18]:
#grid de Hiperparâmetros para serem testados
hyperparameters = {'n_neighbors':range(9,15,2)}
                                
# Grid para k-fold de 10 dobras
knnclf = GridSearchCV(KNeighborsClassifier(), hyperparameters, cv=10, verbose=1, n_jobs=-1, return_train_score=True)

In [19]:
# Treinamento do modelo
%%time
modelknn=knnclf.fit(X_train,y_train)

Fitting 10 folds for each of 3 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


CPU times: user 607 ms, sys: 88.1 ms, total: 695 ms
Wall time: 58 s


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   57.8s finished


In [148]:
modelknn.best_params_

{'n_neighbors': 13}

In [141]:
# Geração da previsão
y_pred=modelknn.predict(X_test)
y_predProb=modelknn.predict_proba(X_test)

In [142]:
# métricas de avaliação do modelo
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.33      0.04      0.08      2763
           1       0.68      0.96      0.80      5955

    accuracy                           0.67      8718
   macro avg       0.51      0.50      0.44      8718
weighted avg       0.57      0.67      0.57      8718



In [ ]:
# Matrix de confusao
print(classification_report(y_test, y_pred, target_names=["Eleito", "Não Eleito"]))

# 2 - Decision Tree
### este método utiliza a estratégia dividir para conquistar, ou seja, um problema complexo é dividido em problemas mais simples, aos quais recursivamente é aplicado a mesma estratégia. As soluções dos subproblemas podem ser combinadas, na forma de uma árvore, para produzir a solução de um problema complexo (Faceli et al., 2015). 

In [99]:
#grid de Hiperparâmetros para serem testados
hyperparameters = {'max_depth':range(11,15),
                   'criterion': ['gini', 'entropy'],
                   }
# Grid para k-fold de 10 dobras
DTreeclf = GridSearchCV(DecisionTreeClassifier(random_state=42), hyperparameters, cv=10, verbose=1, n_jobs=-1, return_train_score=True)

In [100]:
%%time
DTreeModel=DTreeclf.fit(X_train,y_train)

Fitting 10 folds for each of 8 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


CPU times: user 282 ms, sys: 32 ms, total: 314 ms
Wall time: 1.76 s


[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    1.7s finished


In [101]:
DTreeclf.best_params_

{'criterion': 'entropy', 'max_depth': 11}

In [102]:
DTreeclf.best_estimator_

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=11, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [103]:
# Gerando a previsão do DecisionTree
y_predTree=DTreeModel.predict(X_test)

In [104]:
# Gerando as métricas de avaliação
print(metrics.classification_report(y_test, y_predTree))

              precision    recall  f1-score   support

           0       0.23      0.01      0.01      2763
           1       0.68      0.99      0.81      5955

    accuracy                           0.68      8718
   macro avg       0.46      0.50      0.41      8718
weighted avg       0.54      0.68      0.56      8718



In [ ]:
# Matrix de confusao
print(classification_report(y_test, y_predTree, target_names=["Eleito", "Não Eleito"]))

# 3 - Random Forest
### este algorítmo que cria várias árvores de decisão e faz uma combinação (ensemble) entre elas, buscando obter resultados com maior acurácia e maior estabilidade. 

In [120]:
#pipeline para realizar os k-folds no conjunto de treino
pipeline = make_pipeline(preprocessing.StandardScaler(),RandomForestClassifier(n_estimators=200, random_state=42))
#grid de Hiperparâmetros para serem testados
hyperparameters = {'randomforestclassifier__max_features' : ['auto', 'sqrt', 'log2'],
                  'randomforestclassifier__max_depth':[7,11]}
# Grid para k-fold de 10 dobras
clf = GridSearchCV(pipeline, hyperparameters, cv=10)

In [124]:
%%time
#Ajusta o modelo aos dados de treino
modelRandomTree=clf.fit(X_train,y_train)

CPU times: user 1min 30s, sys: 178 ms, total: 1min 30s
Wall time: 1min 30s


In [126]:
modelRandomTree.best_params_

{'randomforestclassifier__max_depth': 7,
 'randomforestclassifier__max_features': 'auto'}

In [127]:
#Previsão
y_predRForest = modelRandomTree.predict(X_test)

In [128]:
# Avaliação
print(metrics.classification_report(y_test, y_predRForest))

              precision    recall  f1-score   support

           0       0.41      0.00      0.01      2763
           1       0.68      1.00      0.81      5955

    accuracy                           0.68      8718
   macro avg       0.55      0.50      0.41      8718
weighted avg       0.60      0.68      0.56      8718



In [ ]:
# Matrix de confusao
print(classification_report(y_test, y_predRForest, target_names=["Eleito", "Não Eleito"]))

# SVM 
### este método busca uma linha de separação entre duas classes distintas analisando os dois pontos, um de cada grupo, mais próximos da outra classe. o SVM escolhe a reta — também chamada de hiperplano em maiores dimensões— entre dois grupos que se distancia mais de cada um. No entanto, existem grupos que não podem ser separados somente por hiperplanos, e são utilizados o SVM não linear para delimitar as duas classes, que traçará uma ou mais linhas retas ou curvas para separar as classes da melhor forma possível.

In [23]:
#grid de Hiperparâmetros para serem testados
hyperparameters = {'kernel':['rbf','poly']} 
# Grid para k-fold de 10 dobras
SVMclf = GridSearchCV(SVC(random_state=42, gamma=0.7, probability=True), hyperparameters, cv=10, verbose=1, n_jobs=-1, return_train_score=True)


In [25]:
SVMclf = SVC(kernel='rbf', gamma=0.7, probability=True, random_state=42, decision_function_shape='ovr')

In [26]:
%%time
modelSVM=SVMclf.fit(X_train,y_train)

CPU times: user 2min 2s, sys: 591 ms, total: 2min 3s
Wall time: 2min 3s


In [ ]:
modelSVM.best_params_

In [28]:
# previsão
y_pred_SVM = modelSVM.predict(X_test)

In [29]:
# Avaliação
print(metrics.classification_report(y_test, y_pred_SVM))

              precision    recall  f1-score   support

           0       0.32      0.00      0.01      2763
           1       0.68      1.00      0.81      5955

    accuracy                           0.68      8718
   macro avg       0.50      0.50      0.41      8718
weighted avg       0.57      0.68      0.56      8718



In [ ]:
# Matrix de confusao
print(classification_report(y_test, y_pred_SVM, target_names=["Eleito", "Não Eleito"]))